In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import re

# Notebook de contôle

## Contrôles à appliquer avant le workflow Dagster

### Moissonnage des sets OAI-PMH de TEF pour classification Dewey des thèses (et sauvegarde)

In [7]:
def  scrapping_oai_sets_dewey():
    url = "https://www.theses.fr/schemas/tef/recommandation/oai_sets.html"
    resp = requests.get(url).text  # ou f = http.request('GET', url).data
    soup = BeautifulSoup(resp, features="lxml")
    oai_list = []
    for row in soup.findAll("table")[0].findAll("tr"):
        label = re.sub('<!--.*-->|\r|\n', '', str(row.findAll("td")[0].get_text(strip=True)), flags=re.DOTALL)
        label = re.sub('\s{2,}|&nbsp;', ' ', label)
        oai_list.append(
            {
                "label": label,
                "code": row.findAll("td")[1].get_text(strip=True),
            }
        )
    df = pd.DataFrame(oai_list[1:])
    df['main_domain'] = df['code'].apply(lambda x: 'Sciences, Technologies, Santé' if ((str(x[4]) == "5") | (str(x[4]) == "6") | (str(x[4:7]) == "004")) else 'Lettres, Sciences Humaines et Sociales')
    return df

In [8]:
scrapping_oai_sets_dewey()

,label,code,main_domain
0,"Informatique, information, généralités",ddc:000,"Lettres, Sciences Humaines et Sociales"
1,Informatique,ddc:004,"Sciences, Technologies, Santé"
2,Bibliothéconomie et sciences de l'information,ddc:020,"Lettres, Sciences Humaines et Sociales"
3,Organisations générales et muséologie,ddc:060,"Lettres, Sciences Humaines et Sociales"
4,"Médias d'information, journalisme, édition",ddc:070,"Lettres, Sciences Humaines et Sociales"
...,...,...,...
93,"Histoire générale de l'Asie, Orient, Extrême-O...",ddc:950,"Lettres, Sciences Humaines et Sociales"
94,Histoire générale de l'Afrique,ddc:960,"Lettres, Sciences Humaines et Sociales"
95,Histoire générale de l'Amérique du Nord,ddc:970,"Lettres, Sciences Humaines et Sociales"
96,Histoire générale de l'Amérique du Sud,ddc:980,"Lettres, Sciences Humaines et Sociales"


In [ ]:
df.to_csv("02_intermediate/oai_set_specs_dewey_labels.csv", index=False, encoding='utf8')

## Divers